In [85]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import qr, eig
from scipy.sparse.linalg import eigsh
from numpy.linalg import norm
import scipy.sparse as sp
import scipy.sparse.linalg as spla
import time
from scipy.linalg import eigh_tridiagonal
import scanpy as sc


In [26]:
np.random.seed(42)
n = 500
density = 0.01  
A_sparse = sp.random(n, n, density=density, format='csr')
A_sparse = 0.5*(A_sparse + A_sparse.T)  
A = A_sparse.toarray()  

In [76]:
adata= sc.read('bcsstk13.mtx')
A=adata.X.toarray()
n=A.shape[0]

In [74]:
adata=sc.read('rdb1250.mtx.gz')
A=adata.X.toarray()
n=A.shape[0]

In [86]:
def lanczos_largest_eigenvalue_householder(A, max_iter=1000, tol=1e-10):

    
    n = A.shape[0]
    # if isinstance(A, csr_matrix):
    #     A = A.tocsr()
    # else:
    #     A = np.array(A)

    V = np.zeros((max_iter + 1, n))
    v = np.random.rand(n)
    v = v / np.linalg.norm(v)
    V[0] = v

    alphas = []
    betas = []

    beta_prev = 0.0
    for j in range(max_iter):
        w = A.dot(V[j]) 
        for i in range(j + 1):
            hj = np.dot(V[i], w)
            w = w - hj * V[i]

        alpha = np.dot(V[j], w)
        alphas.append(alpha)

        w = w - alpha * V[j]

        beta = np.linalg.norm(w)
        betas.append(beta)

        # Check for convergence
        if beta < tol:
            print(f"Converged at iteration {j+1}")
            break


        V[j + 1] = w / beta

    T = np.diag(alphas) + np.diag(betas[1:], k=1) + np.diag(betas[1:], k=-1)
    eigenvalues, _ = eigh_tridiagonal(alphas, betas[1:])
    largest_eigenvalue = eigenvalues[-1]
    eigenvector = V[j] if j < max_iter else V[-1]
    return largest_eigenvalue, eigenvector

In [95]:
t0 = time.time()
vals_lanczos, vecs_lanczos = lanczos_largest_eigenvalue_householder(A,200)
t_lanczos = time.time() - t0

1409489493581.2664

In [103]:
err_lanczos = rel_error(lambda_max_lanczos, lambda_max_ref)
print(err_lanczos)

8.416045527439885e-08


In [ ]:

t0 = time.time()
vals_ref, vecs_ref = np.linalg.eigh(A)
t_ref = time.time() - t0
lambda_max_ref = vals_ref[-1]  
print("Reference (numpy.linalg.eigh) time: {:.6f} s".format(t_ref))
print("Reference max eigenvalue: ", lambda_max_ref)

def rel_error(approx_val, true_val):
    return abs(approx_val - true_val) / (abs(true_val) + 1e-15)

t0 = time.time()
vals_lanczos, vecs_lanczos = spla.eigsh(A, k=1, which='LM')
t_lanczos = time.time() - t0
lambda_max_lanczos = vals_lanczos[-1]
# t0 = time.time()
# vals_lanczos, vecs_lanczos = lanczos_largest_eigenvalue_householder(A,600)
# t_lanczos = time.time() - t0
err_lanczos = rel_error(lambda_max_lanczos, lambda_max_ref)
print("\nLanczos:")
print("  Time: {:.6f} s".format(t_lanczos))
print("  Eigenvalue: {:.6f}, Error: {:.6e}".format(lambda_max_lanczos, err_lanczos))

def power_iteration(A_mat, max_iter=10000, tol=1e-10):
    b = np.random.rand(A_mat.shape[0])
    b = b / np.linalg.norm(b)
    prev_val = 0
    for i in range(max_iter):
        b_new = A_mat.dot(b)
        b_new_norm = np.linalg.norm(b_new)
        b_new = b_new / b_new_norm
        eigenval = b_new_norm
        if abs(eigenval - prev_val) < tol:
            break
        b = b_new
        prev_val = eigenval
    return eigenval, b

t0 = time.time()
lambda_max_power, vec_power = power_iteration(A)
t_power = time.time() - t0
err_power = rel_error(lambda_max_power, lambda_max_ref)
print("\nPower Iteration:")
print("  Time: {:.6f} s".format(t_power))
print("  Eigenvalue: {:.6f}, Error: {:.6e}".format(lambda_max_power, err_power))


t0 = time.time()
vals_arnoldi, vecs_arnoldi = spla.eigs(A, k=1, which='LM')
t_arnoldi = time.time() - t0
lambda_max_arnoldi = np.real(vals_arnoldi[-1])  # eigs可能返回复数类型
err_arnoldi = rel_error(lambda_max_arnoldi, lambda_max_ref)
print("\nArnoldi (using eigs):")
print("  Time: {:.6f} s".format(t_arnoldi))
print("  Eigenvalue: {:.6f}, Error: {:.6e}".format(lambda_max_arnoldi, err_arnoldi))


def jacobi_davidson(A_mat, max_iter=200, tol=1e-10):
    n = A_mat.shape[0]
    v = np.random.rand(n)
    v /= np.linalg.norm(v)
    eigenval = v @ A_mat @ v
    for i in range(max_iter):
        # Rayleigh quotient
        sigma = v @ A_mat @ v
        # 残量
        r = A_mat @ v - sigma * v
        norm_r = np.linalg.norm(r)
        if norm_r < tol:
            return sigma, v
        M = A_mat - sigma * sp.eye(n, n)
        # 使用最简单的GMRES或直接求解
        t_sol, info = spla.cg(M, -r)
        if info != 0:
            v = v + t_sol
        else:
            v = v + t_sol
        v /= np.linalg.norm(v)
        eigenval = v @ A_mat @ v
    return eigenval, v

t0 = time.time()
lambda_max_jd, vec_jd = jacobi_davidson(A)
t_jd = time.time() - t0
err_jd = rel_error(lambda_max_jd, lambda_max_ref)
print("\nJacobi-Davidson (simplified):")
print("  Time: {:.6f} s".format(t_jd))
print("  Eigenvalue: {:.6f}, Error: {:.6e}".format(lambda_max_jd, err_jd))

def rayleigh_quotient_iteration(A_mat, max_iter=200, tol=1e-10):
    n = A_mat.shape[0]
    v = np.random.rand(n)
    v /= np.linalg.norm(v)
    sigma = v @ A_mat @ v
    for i in range(max_iter):
        # Solve (A - sigma I) w = v
        M = A_mat - sigma * np.eye(n)
        w = np.linalg.solve(M, v)
        w /= np.linalg.norm(w)
        new_sigma = w @ A_mat @ w
        if abs(new_sigma - sigma) < tol:
            return new_sigma, w
        sigma = new_sigma
        v = w
    return sigma, v

# 注：RQI对于大型稀疏系统一般需要使用稀疏求解器，但这里直接用np.linalg.solve示意
# 在实际200阶并不算很大，这里问题不大
t0 = time.time()
lambda_max_rqi, vec_rqi = rayleigh_quotient_iteration(A, max_iter=200)
t_rqi = time.time() - t0
err_rqi = rel_error(lambda_max_rqi, lambda_max_ref)
print("\nRayleigh Quotient Iteration:")
print("  Time: {:.6f} s".format(t_rqi))
print("  Eigenvalue: {:.6f}, Error: {:.6e}".format(lambda_max_rqi, err_rqi))


# 6. Divide-and-conquer (使用numpy.linalg.eigh已实现)

# print("\nDivide-and-conquer (numpy.linalg.eigh):")
# print("  Time: {:.6f} s".format(t_ref))
# print("  Eigenvalue: {:.6f}, Error: {:.6e}".format(lambda_max_ref, rel_error(lambda_max_ref, lambda_max_ref)))


# 7. Homotopy方法
def homotopy_method(A_full, steps=10, max_iter=200, tol=1e-8):
    n = A_full.shape[0]
    D = np.diag(np.diag(A_full))  
    v = np.random.rand(n)
    v /= np.linalg.norm(v)
    eigenval = v @ D @ v  # 初始在D上的特征值等于对角元中最大值附近
    for i in range(1, steps+1):
        t = i / steps
        A_t = (1-t)*D + t*A_full
        # 在A_t上做一次幂迭代寻找最大特征值
        # 初始化使用上一次的v
        for k in range(max_iter):
            w = A_t @ v
            w_norm = np.linalg.norm(w)
            w = w / w_norm
            if np.linalg.norm(w - v) < tol:
                v = w
                eigenval = w_norm
                break
            v = w
    return eigenval, v

t0 = time.time()
lambda_max_homo, vec_homo = homotopy_method(A, steps=10)
t_homo = time.time() - t0
err_homo = rel_error(lambda_max_homo, lambda_max_ref)
print("\nHomotopy Method (simplified):")
print("  Time: {:.6f} s".format(t_homo))
print("  Eigenvalue: {:.6f}, Error: {:.6e}".format(lambda_max_homo, err_homo))

# 汇总运行时间与精度比较

print("\nSummary:")
print("Method               | Time (s)    | Eigenvalue        | Rel Error")
print("--------------------------------------------------------------")
print(f"Reference (eigh)     | {t_ref:.6f}  | {lambda_max_ref:.6f} | {rel_error(lambda_max_ref, lambda_max_ref):.6e}")
print(f"Lanczos (eigsh)      | {t_lanczos:.6f}  | {lambda_max_lanczos:.6f} | {err_lanczos:.6e}")
print(f"Power Iteration      | {t_power:.6f}  | {lambda_max_power:.6f} | {err_power:.6e}")
print(f"Arnoldi (eigs)       | {t_arnoldi:.6f}  | {lambda_max_arnoldi:.6f} | {err_arnoldi:.6e}")
print(f"Jacobi-Davidson      | {t_jd:.6f}  | {lambda_max_jd:.6f} | {err_jd:.6e}")
print(f"Rayleigh Quotient    | {t_rqi:.6f}  | {lambda_max_rqi:.6f} | {err_rqi:.6e}")
#print(f"Divide-and-conquer   | {t_ref:.6f}  | {lambda_max_ref:.6f} | {rel_error(lambda_max_ref, lambda_max_ref):.6e}")
print(f"Homotopy             | {t_homo:.6f}  | {lambda_max_homo:.6f} | {err_homo:.6e}")


Reference (numpy.linalg.eigh) time: 2.361783 s
Reference max eigenvalue:  3114812000000.0

Lanczos:
  Time: 0.045033 s
  Eigenvalue: 3114811785216.000000, Error: 5.366504e-01

Power Iteration:
  Time: 21.122966 s
  Eigenvalue: 3114811955545.700195, Error: 2.947667e-08

Arnoldi (using eigs):
  Time: 0.017957 s
  Eigenvalue: 3114814668800.000000, Error: 8.416046e-07

Jacobi-Davidson (simplified):
  Time: 1309.503088 s
  Eigenvalue: 21319502854.024139, Error: 9.931554e-01

Rayleigh Quotient Iteration:
  Time: 1.531169 s
  Eigenvalue: 19838710252.892338, Error: 9.936308e-01

Homotopy Method (simplified):
  Time: 32.435061 s
  Eigenvalue: 34295090376.746922, Error: 9.889897e-01

Summary:
Method               | Time (s)    | Eigenvalue        | Rel Error
--------------------------------------------------------------
Reference (eigh)     | 2.361783  | 3114812047360.000000 | 0.000000e+00
Lanczos (eigsh)      | 0.045033  | 3114811785216.000000 | 5.366504e-01
Power Iteration      | 21.122966  | 